In [1]:
from selenium import webdriver
import pandas as pd
import os
import itertools
import time
import numpy as np
from datetime import datetime

import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
url = "https://www.airasia.com/en/gclassb?gclid=Cj0KCQjwqs3rBRCdARIsADe1pfTS--d9aS2CI6oV__jWRsi9BwHq3SRP_Fbz9xMOsT4yIXPRrCqTMGYaAvGTEALw_wcB&gclsrc=aw.ds"

In [3]:
path = os.getcwd()+'/geckodriver'
driver = webdriver.Firefox(executable_path=path)
driver.get(url)
time.sleep(10)

In [4]:
def getflightplan():
    allplandc = driver.find_elements_by_class_name('product-widget-item')
    nextbutton = driver.find_element_by_class_name('product-widget-arrows.product-widget-arrow-right.ng-star-inserted')
    flight = []
    for i in range(7):    
        try :
            for plan in allplandc:
                if plan.text != '':
                    if 'hotel' not in plan.text and len(plan.text.split('\n'))!=2:
                        flight.append(plan.text)
                else:
                    pass
            nextbutton.click()
            time.sleep(3)
        except Exception as e:
            pass
    
    return flight
    
def cleanflight():
    data = getflightplan()
    keren = []
    for asu in data:
        if 'destination' in asu:
            keren.append(asu.split('\n')[1:])
        elif asu == '':
            pass
        else:
            keren.append(asu.split('\n'))
    df = pd.DataFrame(keren, columns = ['destination','time','price'])
    df = df.drop_duplicates()
    df['departure'] = 'Jakarta'
    
    now = datetime.now()
    date_time = now.strftime("%m-%d-%Y")
    df['datecreated'] = date_time
    return df

def sendemail(receiver):
    subject = "List Promosi Flight AirAsia"
    message = """\
    Subject: Dear Subscriber,

    Berikut terlampir file excel list promo flight AirAsia.
    Silahkan email untuk request list promo lainnya :)

    Terimakasih

    Regards,


    ScrapperHunter"""
    body = message
    sender_email = "bambangmodjo31@gmail.com"
    receiver_email = receiver
    password = "Prabowo.sandi02"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject


    # Add body to email
    message.attach(MIMEText(body, "plain"))

    filename = "/home/husnirama/python_project_dir/List_Promo_AirAsia.xlsx"  # In same directory as script

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    return ("mail sent!")

In [5]:
res = cleanflight()
driver.quit()

In [6]:
url = "https://www.airasiabig.com/my/en/earn-redeem"
driver = webdriver.Firefox(executable_path=r'/home/husnirama/python_project_dir/geckodriver')
driver.get(url)
time.sleep(10)

btn = driver.find_element_by_class_name('button.is-paddingmedium.txt-red.medium09')
btn.click()
time.sleep(5)

In [7]:
def getdf():
    flights = driver.find_elements_by_class_name('has-text-left')
    points = driver.find_elements_by_class_name('has-text-right')
    redeemtime = driver.find_elements_by_class_name('is-hidden-mobile')
    
    start = []
    end =[]
    pts = []
    for plan in flights:
        if plan.text!='':
            start.append(plan.text.split('\n')[0])
            if len(plan.text.split('\n'))>1:
                end.append(plan.text.split('\n')[1])

    for plan in points:
        if plan.text!='':
            pts.append(plan.text.split('\n')[0])
            
    for redeem in redeemtime:
        if 'redemption' in redeem.text.lower():
            rdm = redeem.text.split('|')
    
    df = pd.DataFrame({'destination' : end[1:], 'departurecode' : start[3:], 'pts' : pts})
    df['pts'] = np.where(df['pts'].str[-3:]=='pts', df['pts'].str[:-4], df['pts'])
    
    df['redeemptiontime'] = rdm[0].split(':')[-1][1:-1]
    df['traveltime'] = rdm[1].split(':')[-1][1:-1]
    
    now = datetime.now()
    date_time = now.strftime("%m-%d-%Y")
    df['datecreated'] = date_time
    return df

def getfulldf():
    df = getdf()
    
    iata = pd.read_excel('IATALIST.xlsx')
    iata = iata[iata.columns[1:].tolist()]
    iata.columns = ['City','Country','departurecode']
    
    fulldf = pd.merge(df,iata, on = ['departurecode'], how = 'left')
    return fulldf  

In [8]:
res2 = getfulldf()
driver.quit()
with pd.ExcelWriter('List_Promo_AirAsia.xlsx',index=False) as writer:  # doctest: +SKIP
    res2.to_excel(writer, sheet_name='BIG')
    res.to_excel(writer, sheet_name='DAILY')

In [9]:
sendemail("cynthia.khong@yahoo.com")

'mail sent!'